# Customer Lifetime Value

In [1]:
#Client id
id = 'be1c4e52bb71e0c54b11a26b8e8d59f2'

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
orders = pd.read_csv('../dataset/olist_orders_dataset.csv', delimiter = ',')
customers = pd.read_csv('../dataset/olist_customers_dataset.csv', delimiter = ',')
reviews = pd.read_csv('../dataset/olist_order_reviews_dataset.csv', delimiter = ',')
payment = pd.read_csv('../dataset/olist_order_payments_dataset.csv', delimiter = ',')
order_items = pd.read_csv('../dataset/olist_order_items_dataset.csv', delimiter = ',')
products = pd.read_csv('../dataset/olist_products_dataset.csv', delimiter = ',')

named_items = pd.merge(order_items, products, on='product_id')
named_items = named_items[['order_id', 'price', 'freight_value', 'product_category_name']]

final = pd.merge(orders, customers, on='customer_id')
final = pd.merge(final, reviews, on='order_id')
#final = pd.merge(final, payment, on='order_id')
final = pd.merge(final, named_items, on='order_id')

client = final.loc[(final['customer_id'] == id)]

client = client.drop_duplicates()

client = client[['order_id','order_status', 'customer_city', 'customer_state', 
                 'review_score','price', 'freight_value', 'product_category_name', 
                 'order_delivered_customer_date', 'order_estimated_delivery_date',
                 'order_purchase_timestamp']]

client['order_delivered_customer_date'] = pd.to_datetime(client['order_delivered_customer_date'])
client['order_estimated_delivery_date'] = pd.to_datetime(client['order_estimated_delivery_date'])
client['order_purchase_timestamp'] = pd.to_datetime(client['order_purchase_timestamp'])


client['is_delayed'] = client['order_delivered_customer_date'] > client['order_estimated_delivery_date']
client['time_delay'] = client['order_delivered_customer_date'] - client['order_estimated_delivery_date']
client['time_to_wait'] = client['order_estimated_delivery_date'] - client['order_purchase_timestamp']
client['time_waited'] = client['order_delivered_customer_date'] - client['order_purchase_timestamp']

favorite_category = client.groupby(["product_category_name"]).size().reset_index(name="amount")

client['total'] = client['price'] + client['freight_value'] 

m = client.mean()

print('Customer average review score:', ("%.2f" % m[0]))
print('Average item price:', ("%.2f" % m[1]),'R$')
print('Average freight price:', ("%.2f" % m[2]),'R$')
print('Average delay:', ("%.2f" % (m[3]*100)), '%')

average_order = client.groupby('order_id', as_index=False)['price'].sum()

print('Average order price:', ("%.2f" % average_order.mean()[0]),'R$')

# region analisis
city_average_revenue = final.groupby('customer_city', as_index=False)['price'].mean()
state_average_revenue = final.groupby('customer_state', as_index=False)['price'].mean()

city = city_average_revenue.loc[(city_average_revenue['customer_city'] == client.iloc[0]['customer_city'])]
state = state_average_revenue.loc[(state_average_revenue['customer_state'] == client.iloc[0]['customer_state'])]


print('Client\'s city average revenue by client:', ("%.2f" % (city.iloc[0]['price'])),'R$')
print('Client\'s state average revenue by client:', ("%.2f" % (state.iloc[0]['price'])),'R$')

Customer average review score: 1.00
Average item price: 65.41 R$
Average freight price: 11.02 R$
Average delay: 0.00 %
Average order price: 784.98 R$
Client's city average revenue by client: 107.42 R$
Client's state average revenue by client: 109.58 R$


# Client's most bought categories

In [3]:
favorite_category.head()

,product_category_name,amount
0,cama_mesa_banho,12


In [4]:
city.head()

,customer_city,price
3588,sao paulo,107.415274
